In [97]:
from pandas import Series, DataFrame
import pandas as pd
import nltk
import operator
import re,string
from patsy import dmatrices
%pylab inline

### (1) import file

In [98]:
train = pd.read_csv('/Users/leeanthea/Documents/Train_rev1.csv')

train[:3]

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk


In [99]:
import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')

description  = train['FullDescription'][:1000]

corpus = description.str.cat(sep=' ')

corpus = corpus.decode('utf-8')

### (2) tokenize the corpus

In [100]:
corpus = corpus.replace('/',' ')

corpus = re.sub(r'[^\w\s]','',corpus)

corpus_words = nltk.word_tokenize(corpus.lower())

### (3) Calculate the frequencies of PoS

In [101]:
pos = nltk.pos_tag(corpus_words)

pos[:11]

[(u'engineering', 'NN'),
 (u'systems', 'NNS'),
 (u'analyst', 'NN'),
 (u'dorking', 'VBG'),
 (u'surrey', 'NN'),
 (u'salary', 'NN'),
 (u'k', 'NN'),
 (u'our', 'PRP$'),
 (u'client', 'NN'),
 (u'is', 'VBZ'),
 (u'located', 'VBN')]

In [102]:
cutoff = np.percentile(train[['SalaryNormalized']], 75)
train['category'] = 'high'
train.ix[train.SalaryNormalized < cutoff, 'category'] = 'low'

train['token'] = ""
fullset = train[['FullDescription','token','category']][:1000]

pos_bigram = list(nltk.bigrams(pos))
pattern = [('JJ', 'NN'),('JJ','NNS'),('NN','NNS')]
import copy
corpus_words_bi = copy.deepcopy(corpus_words)

for i in range(len(pos_bigram)):
    (a,b),(c,d) = pos_bigram[i]
    if (b,d) in pattern:
        corpus_words_bi.append(' '.join(corpus_words[i:i+len(pattern)]))

def document_features_bigram(document):
    document_words = set(document)
    features = {}
    for wd in corpus_words_bi:
        features['contains({})'.format(wd)] = (wd in document_words)
    return features

In [103]:


for i in range(len(fullset)):
    words = re.sub(r'[^\w\s]','',fullset.FullDescription[i])
    tokens = nltk.word_tokenize(words.lower().decode('utf-8'))
    tokens = [w for w in tokens if w.isalpha()==True]
    tags = nltk.pos_tag(tokens)
    bigram = list(nltk.bigrams(tags))
    for n in range(len(bigram)):
        (a,b),(c,d) = bigram[n]
        if (b,d) in pattern:
            tokens.append(' '.join(tokens[n:n+len(pattern)]))
    feature = document_features_bigram(tokens)
    fullset.token[i] = feature

fullset['feature_sets'] = zip(fullset.token, fullset.category)
featuresets = fullset.feature_sets.tolist()
train_set, test_set = featuresets[:500], featuresets[500:]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [104]:
nltk.classify.accuracy(classifier, test_set)

0.808